In [1]:
from datasets import load_dataset 
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset('glue', 'mrpc')
checkpoint = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example: dict) -> dict:
    return tokenizer(example['sentence1'], example['sentence2'], truncation = True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched = True)
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

/home/kgbtplus/projects/transformers/transformers/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

In [3]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['validation'],
    data_collator = data_collator,
    tokenizer = tokenizer,
)

/tmp/ipykernel_56449/353238869.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [5]:
trainer.train()

Step,Training Loss
500,0.624000
1000,0.573100


TrainOutput(global_step=1377, training_loss=0.5492067911872504, metrics={'train_runtime': 134.072, 'train_samples_per_second': 82.075, 'train_steps_per_second': 10.271, 'total_flos': 405114969714960.0, 'train_loss': 0.5492067911872504, 'epoch': 3.0})

In [9]:
predictions = trainer.predict(tokenized_datasets['validation'])
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


In [10]:
import numpy as np

preds = np.argmax(predictions.predictions, axis = -1)

In [12]:
import evaluate 

metric = evaluate.load('glue', 'mrpc')
metric.compute(predictions = preds, references = predictions.label_ids)

{'accuracy': 0.8284313725490197, 'f1': 0.8813559322033898}

In [13]:
def compute_metrics(eval_preds: dict) -> dict:
    metrics = evaluate.load('glue', 'mrpc')
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis = -1)
    return metrics.compute(predictions = predictions, references = labels)

In [17]:
training_args = TrainingArguments("test-trainer", evaluation_strategy = "epoch", save_strategy = 'no')
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 2)

trainer = Trainer(
    model,
    training_args,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['validation'],
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics,
)

/home/kgbtplus/projects/transformers/transformers/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_56449/1605950300.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [19]:
trainer.train()
trainer.save_model('test-trainer/model')

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.671140,0.855392,0.896673
2,0.182100,0.686293,0.860294,0.897297
3,0.072400,0.852018,0.857843,0.899306
